<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/prep_data/SST_before_after_storm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import glob as glob

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
year  = '2024'
basin = 'north_atlantic'
time_delta = 5 # days before/after storm
storm = 'MILTON'

if storm == 'BERYL':
  cMin, cMax = [15., 30.]
  diff_cMin, diff_cMax = [-5., 5.]
elif storm == "HELENE":
  cMin, cMax = [25., 32.]
  diff_cMin, diff_cMax = [-2., 2.]
elif storm == "KIRK":
  cMin, cMax = [20., 30.]
  diff_cMin, diff_cMax = [-5., 5.]
elif storm == "MILTON":
  cMin, cMax = [26., 32.]
  diff_cMin, diff_cMax = [-5., 5.]
else:
  cMin, cMax = [-99., 99.]
  diff_cMin, diff_cMax = [-99., 99.]

dPath = f'drive/MyDrive/datasets/hurr/{year}/SAT_datasets/'
fNames = sorted( glob.glob(f'{dPath}/*{storm}*'))

print(f'Found [{len(fNames)}] files for {storm}, names:\n')
for fName in fNames:
  print(fName)

In [ ]:
# track
fName_track = dPath + f'{year}_{basin}_{storm}.nc'
track = xr.open_dataset(fName_track)
#print(track)

vName = 'SST'
# SST (OSTIA)
fName_sat_sst = dPath + f'{vName}_{year}_{storm}.nc'
sat_sst = xr.open_dataset(fName_sat_sst)
#print(sat_sst)

### Plot SST before and after

In [ ]:
plot_width, plot_height, plot_dpi = [16, 6, 120]

fig, ax = plt.subplots(figsize=(plot_width, plot_height), nrows=1, ncols=3, sharey=True, clear=True, dpi=180, subplot_kw={'projection': ccrs.PlateCarree()})

im1a=(sat_sst['analysed_sst']-273.15).isel(time=0).plot(ax=ax[0], vmin=cMin, vmax=cMax, cmap='viridis', add_colorbar=False, transform=ccrs.PlateCarree())
im1a=ax[0].scatter(track['lon'], track['lat'], s=track['vmax'], ls='None', facecolor='None', edgecolor='w', transform=ccrs.PlateCarree())

im2a=(sat_sst['analysed_sst']-273.15).isel(time=-1).plot(ax=ax[1], vmin=cMin, vmax=cMax, cmap='viridis', add_colorbar=False, transform=ccrs.PlateCarree())
im2b=ax[1].scatter(track['lon'], track['lat'], s=track['vmax'], ls='None', facecolor='None', edgecolor='w', transform=ccrs.PlateCarree())
ax[1].set_ylabel('')

im3a=(sat_sst['analysed_sst'].isel(time=-1)-sat_sst['analysed_sst'].isel(time=0)).plot(ax=ax[2], vmin=diff_cMin, vmax=diff_cMax, add_colorbar=False, cmap='RdBu_r', transform=ccrs.PlateCarree())
im3b=ax[2].scatter(track['lon'], track['lat'], s=track['vmax'], ls='None', facecolor='None', edgecolor='k', transform=ccrs.PlateCarree())

ax[2].set_title('SST diff: after - before')
ax[2].set_ylabel('')

cax1 = fig.add_axes([0.635, 0.35, 0.01, 0.3])
cax2 = fig.add_axes([0.91, 0.35, 0.01, 0.3])

cbar1 = plt.colorbar(im2a, cax1)
cbar1.ax.set_title('SST', fontsize=14)
#cbar1.ax.set_ylabel('SST', fontsize=14)
cbar1.ax.set_xlabel('[°C]', fontsize=14)

cbar2 = plt.colorbar(im3a, cax2)
cbar2.ax.set_title('Delta\nSST', fontsize=14)
cbar2.ax.set_xlabel('[°C]', fontsize=14)

for ax_i in ax:
  ax_i.coastlines()
  ax_i.add_feature(cfeature.LAND, facecolor='0.7')

  gl = ax_i.gridlines(draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False
  gl.xformatter = LONGITUDE_FORMATTER
  gl.yformatter = LATITUDE_FORMATTER
  gl.xlabel_style = {'size': 4}
  gl.ylabel_style = {'size': 4}

#plt.tight_layout()

figName = "{}_{}_{}_{}".format(year, basin, storm, vName)
plt.savefig(figName, bbox_inches='tight', dpi=plot_dpi)

In [ ]:
(sat_sst['analysed_sst']-273.15).isel(time=0).plot(cmap='viridis')
#vmin=26, vmax=32)

## SST under the storm track: few days before - after storm passage

In [ ]:
plot_width, plot_height, plot_dpi = [6, 4, 120]

fig = plt.figure(figsize=(plot_width, plot_height), clear=True, dpi=180)
ax = plt.axes()

dts = [-5, -1, 0, 1, 5] # days relative to storm

vals = np.zeros((len(dts), len(track['time'])))
# gather data
for idt, dt in enumerate(dts):
  print(f'\n{idt} Gathering data {dt} days relative to storm')
  for iT in range(len(track['time'])):
    pos_t, pos_x, pos_y = \
     [track.isel(time=iT).time.values+np.timedelta64(dt, 'D'),\
        track.isel(time=iT).lon.values, track.isel(time=iT).lat.values]
    #print(pos_t, pos_x, pos_y)
    vals[idt, iT] = sat_sst.analysed_sst.sel(time=pos_t, longitude=pos_x, latitude=pos_y, method='nearest').values

  ax.plot(track['time'].values, vals[idt,:]-273.15, label=f'{dt} days')
ax.legend()
ax.set_ylabel('SST [°C]')
ax.set_xlabel('Time')
ax.tick_params(axis='x', rotation=45)

ax2 = ax.twinx()
ax2.plot(track['time'].values, track['mslp'].values, marker='o',\
         ls='--', color='k')
ax2.set_ylabel('MSLP [hPa]')

ax2.text(track.isel(time=-6)['time'].values, track.mslp.min()+10,\
         '{}'.format(storm), c='r', fontsize=16)
figName = "along_track_{}_{}_{}_{}.png".format(vName, year, basin, storm)
plt.savefig(figName, bbox_inches='tight')